# Getting data from facebook Graph API

In [47]:
!pip install facebook-sdk --quiet

In [62]:
import json
import facebook

def main():
    token = {"EAAOpoXPN6PIBAGyF8LO9nftNjvXQiwe6zbj99RFUcWFBJhD7SwrMQkvZBzVKSQHsqYT59TbrKWJlECP3dWgrr1j1u4jOay7SYHQWmWxMAg6xNWUknOpydVvcuj1kqySqscZCb8ZCZB7ZAWIhXmve8V4ZB3DgyACMkNwMZA9QJia7QhAxBdwZCOdfrPz3ygqZAHlxUgmQAatm8FAdYgJ5PaIrBev50hUZChogrSow93ldF1EiIj3FLcrKE8ZARAH5ht5ShQZD"}
    graph = facebook.GraphAPI(token)
    
    fields = ['id','name','posts']
    profile = graph.get_object("me",fields = fields)
    print(type(profile))
    all_data = json.dumps(profile,indent=4)
    print(json.dumps(profile,indent=4))
    return profile
    

In [63]:
all_data = main()

<class 'dict'>
{
    "posts": {
        "data": [
            {
                "created_time": "2022-04-19T12:53:08+0000",
                "message": "I am feeling nice, 10 days to go for something great.",
                "id": "530216495379155_538226841244787"
            },
            {
                "created_time": "2022-04-19T12:52:51+0000",
                "message": "You need to talk about something when you are 110% sure that you are going to do it, else do not mention.",
                "id": "530216495379155_538226727911465"
            },
            {
                "created_time": "2022-04-19T12:52:23+0000",
                "message": "The world cubing association is a good organization and it promotes growing of communities.",
                "id": "530216495379155_538226557911482"
            },
            {
                "created_time": "2022-04-19T12:51:55+0000",
                "message": "I liked the speed cube that I bought yesterday.",
                "id":

In [64]:
type(all_data)

dict

In [74]:
json_data = json.dumps(all_data,indent=4)
print(type(json_data))
# print(json_data)
json_data_parsable = json.loads(json_data)
all_posts = json_data_parsable["posts"]["data"]
print(type(all_posts[0]))
print(all_posts)
posts_fetched = []
for i in all_posts:
    posts_fetched.append(i["message"])

<class 'str'>
<class 'dict'>
[{'created_time': '2022-04-19T12:53:08+0000', 'message': 'I am feeling nice, 10 days to go for something great.', 'id': '530216495379155_538226841244787'}, {'created_time': '2022-04-19T12:52:51+0000', 'message': 'You need to talk about something when you are 110% sure that you are going to do it, else do not mention.', 'id': '530216495379155_538226727911465'}, {'created_time': '2022-04-19T12:52:23+0000', 'message': 'The world cubing association is a good organization and it promotes growing of communities.', 'id': '530216495379155_538226557911482'}, {'created_time': '2022-04-19T12:51:55+0000', 'message': 'I liked the speed cube that I bought yesterday.', 'id': '530216495379155_538226371244834'}, {'created_time': '2022-04-19T12:51:28+0000', 'message': 'The cube that I got is horrible, it does not turn well and it does not have corner cutting or edge adjustments', 'id': '530216495379155_538226167911521'}, {'created_time': '2022-04-19T12:50:48+0000', 'message'

KeyError: 'message'

# Sentiment analysis on my posts using Feed Forward Neural Network

In [29]:
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm,tqdm_notebook

In [11]:
# DATA_SET = './datasets/aclimdb.csv'
DATA_SET = './datasets/IMDBDataset.csv'
test =pd.read_csv(DATA_SET).sample(5)
test

,review,sentiment
20560,"I just finished ""Dark Chamber"" aka ""Under Surv...",negative
37402,One of Keli McCarthy's best. This movie is fil...,positive
32381,This movie begins with a man who appears to be...,negative
30984,"Over the years, we've seen a lot of prepostero...",positive
34598,Saw this film ran in the wee hours on TCM. Sev...,negative


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
sentiment_label = le.fit_transform(test["sentiment"])
print(le.classes_)
test_new =test.drop("sentiment",axis="columns")
test_new["sentiment"] = sentiment_label
test_new
# negative - 0 , positive - 1

['negative' 'positive']


,review,sentiment
20560,"I just finished ""Dark Chamber"" aka ""Under Surv...",0
37402,One of Keli McCarthy's best. This movie is fil...,1
32381,This movie begins with a man who appears to be...,0
30984,"Over the years, we've seen a lot of prepostero...",1
34598,Saw this film ran in the wee hours on TCM. Sev...,0


In [13]:
test_vectorizer = CountVectorizer(stop_words='english', max_df = 0.99 , min_df = 0.005)
test_seq = test_vectorizer.fit_transform(test_new['review'].tolist())
#print(test_seq)


In [14]:
class Sequences(Dataset):
    def __init__(self,path):
        df = pd.read_csv(path)
        le = preprocessing.LabelEncoder()
        sentiment_labels = le.fit_transform(df["sentiment"])
        #print(le.classes_)
        df =df.drop("sentiment",axis="columns")
        df["sentiment"] = sentiment_labels

        self.vectorizer = CountVectorizer(stop_words='english', max_df = 0.99 , min_df = 0.005)
        self.sequences = self.vectorizer.fit_transform(df.review.tolist())
        self.labels = df.sentiment.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token , idx in self.token2idx.items()}
    def __getitem__(self,i):
        return self.sequences[i, :].toarray(), self.labels[i]
    def __len__(self):
        return self.sequences.shape[0]        

In [15]:
trial_db = Sequences(DATA_SET)
train_loader = DataLoader(trial_db, batch_size = 4096)
#print(trial_db.vectorizer)
# print(trial_db.sequences)
#print(trial_db.token2idx)

In [16]:
print(trial_db.sequences)

  (0, 2114)	1
  (0, 1628)	1
  (0, 2800)	2
  (0, 1398)	2
  (0, 831)	2
  (0, 1517)	3
  (0, 1240)	1
  (0, 2122)	2
  (0, 857)	1
  (0, 1167)	1
  (0, 304)	6
  (0, 2583)	1
  (0, 2447)	2
  (0, 2192)	1
  (0, 2759)	4
  (0, 2247)	1
  (0, 2857)	2
  (0, 2663)	1
  (0, 1193)	1
  (0, 1990)	1
  (0, 763)	1
  (0, 2252)	1
  (0, 447)	1
  (0, 2720)	1
  (0, 347)	1
  :	:
  (49999, 257)	1
  (49999, 1106)	1
  (49999, 1691)	3
  (49999, 2866)	1
  (49999, 2393)	1
  (49999, 928)	1
  (49999, 2661)	1
  (49999, 397)	1
  (49999, 2176)	1
  (49999, 1429)	1
  (49999, 2700)	1
  (49999, 1476)	1
  (49999, 874)	1
  (49999, 206)	1
  (49999, 1174)	1
  (49999, 1728)	1
  (49999, 1301)	1
  (49999, 929)	1
  (49999, 166)	1
  (49999, 2649)	1
  (49999, 2087)	1
  (49999, 832)	1
  (49999, 344)	1
  (49999, 1107)	1
  (49999, 2085)	1


In [17]:
print(trial_db.sequences[0].shape)

(1, 2891)


In [18]:
print(len(trial_db.labels))
print(trial_db.labels)

50000
[1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 

In [19]:
print(len(trial_db.token2idx))
print(trial_db.token2idx)

2891
{'reviewers': 2114, 'mentioned': 1628, 'watching': 2800, 'just': 1398, 'episode': 831, 'll': 1517, 'hooked': 1240, 'right': 2122, 'exactly': 857, 'happened': 1167, 'br': 304, 'thing': 2583, 'struck': 2447, 'scenes': 2192, 'violence': 2759, 'set': 2247, 'word': 2857, 'trust': 2663, 'hearted': 1193, 'pulls': 1990, 'drugs': 763, 'sex': 2252, 'classic': 447, 'use': 2720, 'called': 347, 'given': 1092, 'security': 2218, 'state': 2401, 'focuses': 1008, 'mainly': 1566, 'city': 442, 'section': 2217, 'prison': 1950, 'glass': 1096, 'face': 902, 'high': 1213, 'em': 795, 'home': 1236, 'irish': 1349, 'death': 632, 'far': 932, 'away': 210, 'say': 2181, 'main': 1565, 'appeal': 148, 'fact': 906, 'goes': 1100, 'shows': 2283, 'wouldn': 2870, 'dare': 614, 'forget': 1026, 'pretty': 1940, 'pictures': 1871, 'mainstream': 1567, 'audiences': 200, 'charm': 409, 'romance': 2144, 'doesn': 725, 'mess': 1631, 'saw': 2180, 'nasty': 1712, 'surreal': 2510, 'couldn': 550, 'ready': 2032, 'watched': 2799, 'developed

In [20]:
print(len(trial_db.idx2token))
print(trial_db.idx2token)

2891
{2114: 'reviewers', 1628: 'mentioned', 2800: 'watching', 1398: 'just', 831: 'episode', 1517: 'll', 1240: 'hooked', 2122: 'right', 857: 'exactly', 1167: 'happened', 304: 'br', 2583: 'thing', 2447: 'struck', 2192: 'scenes', 2759: 'violence', 2247: 'set', 2857: 'word', 2663: 'trust', 1193: 'hearted', 1990: 'pulls', 763: 'drugs', 2252: 'sex', 447: 'classic', 2720: 'use', 347: 'called', 1092: 'given', 2218: 'security', 2401: 'state', 1008: 'focuses', 1566: 'mainly', 442: 'city', 2217: 'section', 1950: 'prison', 1096: 'glass', 902: 'face', 1213: 'high', 795: 'em', 1236: 'home', 1349: 'irish', 632: 'death', 932: 'far', 210: 'away', 2181: 'say', 1565: 'main', 148: 'appeal', 906: 'fact', 1100: 'goes', 2283: 'shows', 2870: 'wouldn', 614: 'dare', 1026: 'forget', 1940: 'pretty', 1871: 'pictures', 1567: 'mainstream', 200: 'audiences', 409: 'charm', 2144: 'romance', 725: 'doesn', 1631: 'mess', 2180: 'saw', 1712: 'nasty', 2510: 'surreal', 550: 'couldn', 2032: 'ready', 2799: 'watched', 680: 'deve

In [21]:
print(trial_db[5][0].shape)

(1, 2891)


In [22]:
# feed forward 1 : bow neural network 
class BoWFF1(nn.Module):
    def __init__(self,vocab_size,hidden1,hidden2):
        super(BoWFF1,self).__init__()
        self.fc1 = nn.Linear(vocab_size,hidden1)
        self.fc2 = nn.Linear(hidden1,hidden2)
        self.fc3 = nn.Linear(hidden2,1)
        
    def forward(self,inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [23]:
model_ff_bow = BoWFF1(len(trial_db.idx2token),128,64)
model_ff_bow

BoWFF1(
  (fc1): Linear(in_features=2891, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [24]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model_ff_bow.parameters() if p.requires_grad],lr=0.001)

In [30]:
model_ff_bow.train()
train_losses = [] 
for epoch in range(10):
    progress_bar = tqdm_notebook(train_loader,leave=False)
    losses = []
    total = 0
    for inputs,target in progress_bar:
        model_ff_bow.zero_grad()
        
        output = model_ff_bow(inputs)
        loss = criterion(output.squeeze(), target.float())
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model_ff_bow.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

/tmp/ipykernel_6101/1782961903.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(train_loader,leave=False)


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch #1	Train Loss: 0.257


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch #2	Train Loss: 0.246


...

Epoch #8	Train Loss: 0.222


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch #9	Train Loss: 0.217


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch #10	Train Loss: 0.213


In [42]:
def predict_sentiment(text):
    model_ff_bow.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor(trial_db.vectorizer.transform([text]).toarray())

        output = model_ff_bow(test_vector)
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            return [f'{prediction:0.3}','Positive sentiment']
        else:
            return [f'{prediction:0.3}','Negative sentiment']
            

In [75]:
posts_fetched

['I am feeling nice, 10 days to go for something great.',
 'You need to talk about something when you are 110% sure that you are going to do it, else do not mention.',
 'The world cubing association is a good organization and it promotes growing of communities.',
 'I liked the speed cube that I bought yesterday.',
 'The cube that I got is horrible, it does not turn well and it does not have corner cutting or edge adjustments',
 "The seven by seven rubik's cube is an interesting puzzle.",
 'The cubing competition is in some days and I am excited for it.',
 'The movie that I watched was a great one.',
 'This is my second post and I am feeling nice right now.',
 'This is my first post , hope I keep posting']

In [76]:
results=[]
for i in posts_fetched :
    results.append([i,predict_sentiment(i)[0],predict_sentiment(i)[1]])

In [77]:
results

[['I am feeling nice, 10 days to go for something great.',
  '0.728',
  'Positive sentiment'],
 ['You need to talk about something when you are 110% sure that you are going to do it, else do not mention.',
  '0.429',
  'Negative sentiment'],
 ['The world cubing association is a good organization and it promotes growing of communities.',
  '0.654',
  'Positive sentiment'],
 ['I liked the speed cube that I bought yesterday.',
  '0.647',
  'Positive sentiment'],
 ['The cube that I got is horrible, it does not turn well and it does not have corner cutting or edge adjustments',
  '0.303',
  'Negative sentiment'],
 ["The seven by seven rubik's cube is an interesting puzzle.",
  '0.619',
  'Positive sentiment'],
 ['The cubing competition is in some days and I am excited for it.',
  '0.46',
  'Negative sentiment'],
 ['The movie that I watched was a great one.', '0.657', 'Positive sentiment'],
 ['This is my second post and I am feeling nice right now.',
  '0.552',
  'Positive sentiment'],
 ['Th

In [78]:
results_df = pd.DataFrame(results,columns=['Post','Prediction Confidence','Sentiment'])

In [79]:
results_df

,Post,Prediction Confidence,Sentiment
0,"I am feeling nice, 10 days to go for something...",0.728,Positive sentiment
1,You need to talk about something when you are ...,0.429,Negative sentiment
2,The world cubing association is a good organiz...,0.654,Positive sentiment
3,I liked the speed cube that I bought yesterday.,0.647,Positive sentiment
4,"The cube that I got is horrible, it does not t...",0.303,Negative sentiment
5,The seven by seven rubik's cube is an interest...,0.619,Positive sentiment
6,The cubing competition is in some days and I a...,0.46,Negative sentiment
7,The movie that I watched was a great one.,0.657,Positive sentiment
8,This is my second post and I am feeling nice r...,0.552,Positive sentiment
9,"This is my first post , hope I keep posting",0.524,Positive sentiment
